# OAM Component Contribution - Simple DynamoDB Example

This notebook demonstrates how to contribute a new OAM component using a simplified DynamoDB table as an example.

## What We'll Create

1. **Crossplane XRD** - Define the custom resource schema
2. **Crossplane Composition** - Define how the resource is composed
3. **KubeVela ComponentDefinition** - Make it usable in KubeVela applications
4. **Test Application** - Verify everything works

## Prerequisites

- Cluster set up from `00_Env-setup.ipynb`
- Crossplane installed with AWS provider
- KubeVela installed

## Step 1: 
## Define the Parameters (API First) for OUR DynamoDB Table
## Define a Crossplane Composition Resource Definition (XRD)
## Deploy it

This XRD defines API for Guidewire DynamoDB table:
- Table name
- Hash key (partition key)
- Attributes
- Region

You could also expose other fields but set them up with sensible defaults and potentially hard code fields that are not modifiable in your platform (**Opinionated Platform**).


In [ ]:
%%bash
# Create directory for Crossplane resources
mkdir -p crossplane/dynamodb

# Create simplified XRD using v2 API
cat > crossplane/dynamodb/xrd.yaml << 'EOF'
apiVersion: apiextensions.crossplane.io/v2
kind: CompositeResourceDefinition
metadata:
  name: xdynamodbtables.demo.kubecon.io
spec:
  scope: Cluster  # Cluster-scoped resources (v1 compatible behavior)
  group: demo.kubecon.io
  names:
    kind: XDynamoDBTable
    plural: xdynamodbtables
  versions:
    - name: v1alpha1
      served: true
      referenceable: true
      schema:
        openAPIV3Schema:
          type: object
          properties:
            spec:
              type: object
              properties:
                name:
                  type: string
                  description: "Name of the DynamoDB table"
                region:
                  type: string
                  description: "AWS region"
                hashKey:
                  type: string
                  description: "Hash (partition) key attribute name"
                attributes:
                  type: array
                  description: "Attribute definitions"
                  items:
                    type: object
                    properties:
                      name:
                        type: string
                      type:
                        type: string
                        enum: ["S", "N", "B"]
                tags:
                  type: object
                  additionalProperties:
                    type: string
                  description: "AWS resource tags"
              required:
                - name
                - region
                - hashKey
                - attributes
            status:
              type: object
              properties:
                tableArn:
                  type: string
                tableId:
                  type: string
EOF

echo "✓ XRD file created at crossplane/dynamodb/xrd.yaml (using v2 API with Cluster scope)"
echo ""

echo "Apply the XRD"
kubectl config use-context k3d-kubecon-NA25

kubectl apply -f crossplane/dynamodb/xrd.yaml

echo ""
echo "Waiting for XRD to be established..."
sleep 3

echo ""
echo "✓ Verifying XRD creation:"
kubectl get xrd xdynamodbtables.demo.kubecon.io


## Step 2: 
## Define HOW to create a DynamoDB table using the above parameters
## Create a Crossplane Composition
## Deploy it

The Composition is an implementation/template that defines how to create the actual DynamoDB table from our XRD.

In [ ]:
%%bash
cat > crossplane/dynamodb/composition.yaml << 'EOF'
apiVersion: apiextensions.crossplane.io/v1
kind: Composition
metadata:
  name: dynamodb-table.demo.kubecon.io
spec:
  compositeTypeRef:
    apiVersion: demo.kubecon.io/v1alpha1
    kind: XDynamoDBTable
  
  mode: Pipeline
  pipeline:
    - step: create-table
      functionRef:
        name: function-patch-and-transform
      input:
        apiVersion: pt.fn.crossplane.io/v1beta1
        kind: Resources
        resources:
          - name: table
            base:
              apiVersion: dynamodb.aws.upbound.io/v1beta1
              kind: Table
              spec:
                forProvider:
                  billingMode: PAY_PER_REQUEST
            
            patches:
              # Patch table name with tenant-atlantis prefix
              - type: FromCompositeFieldPath
                fromFieldPath: spec.name
                toFieldPath: metadata.name
              
              # Patch region
              - type: FromCompositeFieldPath
                fromFieldPath: spec.region
                toFieldPath: spec.forProvider.region
              
              # Patch hash key
              - type: FromCompositeFieldPath
                fromFieldPath: spec.hashKey
                toFieldPath: spec.forProvider.hashKey
              
              # Patch attributes
              - type: FromCompositeFieldPath
                fromFieldPath: spec.attributes
                toFieldPath: spec.forProvider.attribute
              
              # Patch tags
              - type: FromCompositeFieldPath
                fromFieldPath: spec.tags
                toFieldPath: spec.forProvider.tags
              
              # Patch status back
              - type: ToCompositeFieldPath
                fromFieldPath: status.atProvider.arn
                toFieldPath: status.tableArn
              
              - type: ToCompositeFieldPath
                fromFieldPath: status.atProvider.id
                toFieldPath: status.tableId
EOF

echo "✓ Composition file created at crossplane/dynamodb/composition.yaml"
echo ""

kubectl config use-context k3d-kubecon-NA25

echo "Checking Crossplane version..."
kubectl get provider.pkg.crossplane.io -A 2>/dev/null | head -5

echo ""
echo "Applying Composition..."
if kubectl apply -f crossplane/dynamodb/composition.yaml 2>&1 | tee /tmp/composition-apply.log; then
    echo ""
    echo "✓ Composition applied successfully"
else
    echo ""
    echo "❌ Composition apply failed. Checking error..."
    cat /tmp/composition-apply.log
    exit 1
fi

echo ""
echo "Waiting for Composition to be ready..."
for i in {1..10}; do
    if kubectl get composition dynamodb-table.demo.kubecon.io &>/dev/null; then
        echo "✓ Composition is ready"
        break
    fi
    echo "  Waiting... ($i/10)"
    sleep 2
done

echo ""
echo "✓ Verifying Composition creation:"
kubectl get composition dynamodb-table.demo.kubecon.io || echo "⚠ Composition not found - check the error above"

## Step 3: 
## Test Crossplane resources

In [ ]:
%%bash
mkdir -p test

cat > test/test-table.yaml << 'EOF'
apiVersion: demo.kubecon.io/v1alpha1
kind: XDynamoDBTable
metadata:
  name: tenant-atlantis-test-simple-table
  namespace: default
spec:
  name: tenant-atlantis-test-simple-table
  region: us-west-2
  hashKey: id
  attributes:
    - name: id
      type: "S"
  tags:
    "gwcp:v1:dept": "000"
    "gwcp:v1:provisioned-resource:created-by": "kubecon-NA25"
    "gwcp:v1:quadrant:name": "dev"
    "gwcp:v1:resource-type:managed-by": "pod-atlantis"
    "gwcp:v1:resource-type:managed-tool": "crossplane"
    "gwcp:v1:star-system:name": "kubecon"
    "gwcp:v1:tenant:name": "atlantis"
    "gwcp:v1:tenant:app-name": "test-app"
  crossplane:
    compositionRef:
      name: dynamodb-table.demo.kubecon.io
EOF

echo "✓ Test composite resource created with tenant-atlantis prefix"

kubectl config use-context k3d-kubecon-NA25

echo "Applying test composite resource..."
kubectl apply -f test/test-table.yaml

echo ""
echo "Waiting a few seconds for reconciliation..."
sleep 5

echo ""
echo "✓ Checking composite resource:"
kubectl get xdynamodbtables -n default

echo ""
echo "✓ Checking underlying DynamoDB Table resource:"
kubectl get table -n default 2>/dev/null || echo "Table resource not created yet (AWS provider may not be configured)"


## Step 4:

## Create an Application level definition of our DynamoDB table
## Create KubeVela ComponentDefinition (CUE): a reusable component type that applications can use. 
###  A "building block" for applications.
## Deploy it

This makes the DynamoDB table available as a KubeVela component.

In [ ]:
%%bash
mkdir -p kubevela/components/dynamodb

cat > kubevela/components/dynamodb/dynamodb.cue << 'EOF'
"simple-dynamodb": {
    type: "component"
    description: "A basic DynamoDB table component for KubeCon NA 2025."
    attributes: {
        workload: definition: {
            apiVersion: "demo.kubecon.io/v1alpha1"
            kind:       "XDynamoDBTable"
        }
        status: {
            healthPolicy: #"""
                isHealth: bool | *false
                if context.output.status != _|_ {
                    if context.output.status.conditions != _|_ {
                        for c in context.output.status.conditions {
                            if c.type == "Ready" && c.status == "True" {
                                isHealth: true
                            }
                        }
                    }
                }
                """#
            customStatus: #"""
                message: string | *"Provisioning table..."
                if context.output.status != _|_ {
                    if context.output.status.tableArn != _|_ {
                        message: "Table ARN: " + context.output.status.tableArn
                    }
                }
                """#
        }
    }
}

template: {
    output: {
        apiVersion: "demo.kubecon.io/v1alpha1"
        kind:       "XDynamoDBTable"
        metadata: {
            name:      "tenant-atlantis-" + parameter.name
            namespace: context.namespace
        }
        spec: {
            name:       "tenant-atlantis-" + parameter.name
            region:     parameter.region
            hashKey:    parameter.hashKey
            attributes: parameter.attributes
            tags: {
                "gwcp:v1:dept":                            "000"
                "gwcp:v1:provisioned-resource:created-by": "kubecon-NA25"
                "gwcp:v1:quadrant:name":                   "dev"
                "gwcp:v1:resource-type:managed-by":        "pod-atlantis"
                "gwcp:v1:resource-type:managed-tool":      "crossplane"
                "gwcp:v1:star-system:name":                "kubecon"
                "gwcp:v1:tenant:name":                     "atlantis"
                "gwcp:v1:tenant:app-name":                 context.appName
            }
            crossplane: {
                compositionRef: {
                    name: "dynamodb-table.demo.kubecon.io"
                }
            }
        }
    }

    parameter: {
        // +usage=Name of the DynamoDB table (will be prefixed with tenant-atlantis-)
        name: string
        
        // +usage=AWS region default to us-west-2
        region: *"us-west-2" | string
        
        // +usage=Hash key attribute name
        hashKey: string
        
        // +usage=Attribute definitions
        attributes: [...{
            // +usage=Attribute name
            name: string
            // +usage=Attribute type (S=String, N=Number, B=Binary)
            type: "S" | "N" | "B"
        }]
    }
}
EOF

echo "✓ KubeVela ComponentDefinition CUE file created with expected prefix, tags, and Crossplane v2 compositionRef"
echo ""

kubectl config use-context k3d-kubecon-NA25

if command -v vela &> /dev/null; then
    echo "Using vela CLI to apply ComponentDefinition..."
    cd kubevela/components/dynamodb
    vela def apply dynamodb.cue
    echo ""
    echo "✓ ComponentDefinition applied"
else
    echo "⚠ vela CLI not found. Please install it and try again."
    exit 1
fi

echo ""
echo "Checking for ComponentDefinition:"
kubectl get componentdefinition simple-dynamodb -n vela-system 2>/dev/null || echo "Not installed yet"

## Step 5: 
## Create KubeVela Application (**SSSSSS**)
## Deploy the KubeVela Application

In [ ]:
%%bash
cat > test/app.yaml << 'EOF'
apiVersion: core.oam.dev/v1beta1
kind: Application
metadata:
  name: my-dynamodb-app
  namespace: default
spec:
  components:
    - name: users-table
      type: simple-dynamodb
      properties:
        name: users-table
        hashKey: userId
        region: us-west-2
        attributes:
          - name: userId
            type: "S"
EOF

echo "✓ KubeVela application manifest created"

kubectl config use-context k3d-kubecon-NA25

# Only apply if ComponentDefinition exists
if kubectl get componentdefinition simple-dynamodb -n vela-system &>/dev/null; then
    echo "Applying KubeVela application..."
    vela up -f test/app.yaml
    
    echo ""
    sleep 3
    
    echo "✓ Application status:"
    vela status my-dynamodb-app
else
    echo "⚠ ComponentDefinition not found. Skipping application deployment."
    echo "To deploy: Install vela CLI and run the ComponentDefinition cell first."
fi

## Step 6: 
## Verify it all

Check the status of all created resources.

In [ ]:
%%bash
echo "=== Resource Status ==="
echo ""
kubectl config use-context k3d-kubecon-NA25

echo "1. XRD:"
kubectl get xrd xdynamodbtables.demo.kubecon.io

echo ""
echo "2. Composition:"
kubectl get composition dynamodb-table.demo.kubecon.io

echo ""
echo "3. Composite Resources:"
kubectl get xdynamodbtables -A

echo ""
echo "4. DynamoDB Tables (Managed Resources):"
kubectl get table -A 2>/dev/null || echo "No tables found (AWS provider may not be configured)"

echo ""
echo "5. ComponentDefinition:"
kubectl get componentdefinition simple-dynamodb -n vela-system 2>/dev/null || echo "Not installed"

echo ""
echo "6. KubeVela Applications:"
vela ls -A 2>/dev/null || echo "No applications"
vela status my-dynamodb-app

nohup vela show --web  simple-dynamodb > /dev/null 2>&1 &


## Cleanup

To clean up all resources created in this demo, use the dedicated cleanup notebook:

📓 **`01-OAM-cleanup.ipynb`**

The cleanup notebook will:
- Delete KubeVela Applications
- Delete Composite Resources
- Wait for managed resources cleanup
- Delete ComponentDefinition
- Delete Composition
- Delete XRD
- Verify cleanup completion

## Summary

In this notebook, we demonstrated a simplified OAM component contribution workflow:

1. ✓ Created a simplified **Crossplane XRD** with minimal fields
2. ✓ Created a **Composition** to define resource creation
3. ✓ Created a **KubeVela ComponentDefinition** (CUE file)
4. ✓ Tested with a **Composite Resource**
5. ✓ Created a **KubeVela Application** example
6. ✓ Provided **publishing guidelines** and metadata

### Key Simplifications

This simplified version includes only:
- Table name
- Region
- Hash key
- Basic attributes
- Pay-per-request billing

The full production version would include:
- Range keys, TTL, GSI/LSI
- Encryption, backup settings
- Tags and governance metadata
- Deletion protection

### Next Steps

1. Extend the schema with additional DynamoDB features
2. Add validation and default values
3. Create comprehensive tests

### Cleanup

When you're done, run the **`01-OAM-cleanup.ipynb`** notebook to remove all resources.

## Summary

In this notebook, we demonstrated a simplified OAM component contribution workflow:

1. ✓ Created a simplified **Crossplane XRD** with minimal fields
2. ✓ Created a **Composition** to define resource creation
3. ✓ Created a **KubeVela ComponentDefinition** (CUE file)
4. ✓ Tested with a **Composite Resource**
5. ✓ Created a **KubeVela Application** example
6. ✓ Provided **publishing guidelines** and metadata

### Key Simplifications

This simplified version includes only:
- Table name
- Region
- Hash key
- Basic attributes
- Pay-per-request billing

The full production version would include:
- Range keys, TTL, GSI/LSI
- Encryption, backup settings
- Tags and governance metadata
- Deletion protection

### Next Steps

1. Extend the schema with additional DynamoDB features
2. Add validation and default values
3. Create comprehensive tests